In [77]:
import os
import json
from datetime import datetime
from dotenv import load_dotenv
import openai
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

# Load environment variables
load_dotenv()

# Get the API key from environment
api_key = os.getenv('API_KEY')
if not api_key:
    raise ValueError("API_KEY not found in .env file. Please add your OpenAI API key to the .env file.")

# Create OpenAI client
client = openai.OpenAI(api_key=api_key)

# Excel file path
EXCEL_FILE = os.path.join(os.path.dirname(os.path.abspath('__file__')), 'transactions.xlsx')

# Create Excel file if it doesn't exist
if not os.path.exists(EXCEL_FILE):
    # Create a new DataFrame with the required columns
    df = pd.DataFrame(columns=['Date', 'Description', 'Amount', 'Category', 'Type'])
    # Save it to Excel
    df.to_excel(EXCEL_FILE, index=False, sheet_name='Expenses')

## Excel File Structure

#The transactions are stored in a local Excel file named `transactions.xlsx` with the following columns:
1. Date (YYYY-MM-DD format)
2. Description (what the transaction was for)
3. Amount (positive number)
4. Category (e.g., Food, Transport, Salary, etc.)
5. Type (Income or Expense)

#The file is automatically created in the same folder as this notebook if it doesn't exist.

In [78]:
def parse_transaction(text):
    """Use OpenAI to parse transaction details from natural language input"""
    current_date = datetime.now().strftime('%Y-%m-%d')
    
    prompt = f"""
    Parse the following financial transaction(s) into a structured format. If multiple transactions are mentioned, return a list of JSON objects.
    Text: "{text}"
    
    Extract and return JSON with these fields for each transaction:
    - date: string in YYYY-MM-DD format (if no date is mentioned, use "{current_date}")
    - amount: number (positive)
    - description: string
    - category: string (e.g., Food, Transport, Salary, etc.)
    - type: string (either "Income" or "Expense")
    
    Examples:
    Input: "on July 20th paid $50 for internet"
    Output: {{"date": "2025-07-20", "amount": 50, "description": "internet", "category": "Utilities", "type": "Expense"}}
    
    Input: "$5 on groceries and $20 on lunch"
    Output: [
        {{"date": "{current_date}", "amount": 5, "description": "groceries", "category": "Food", "type": "Expense"}},
        {{"date": "{current_date}", "amount": 20, "description": "lunch", "category": "Food", "type": "Expense"}}
    ]
    
    Input: "yesterday spent $30 on gas and today $15 on coffee"
    Output: [
        {{"date": "2025-07-23", "amount": 30, "description": "gas", "category": "Transport", "type": "Expense"}},
        {{"date": "2025-07-24", "amount": 15, "description": "coffee", "category": "Food", "type": "Expense"}}
    ]
    """
    
    try:
        # Use the global client instance
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": f"You are a financial transaction parser. Respond only with valid JSON. For relative dates like 'yesterday' or 'last week', convert them to actual dates based on the current date being {current_date}. If multiple transactions are mentioned, return a list of JSON objects. If no date is mentioned, use {current_date}."},
                {"role": "user", "content": prompt}
            ]
        )
        
        parsed = json.loads(response.choices[0].message.content)
        # If single transaction is returned, convert to list
        if isinstance(parsed, dict):
            return [parsed]
        return parsed
    except Exception as e:
        print(f"Error calling OpenAI API: {str(e)}")
        return None

In [79]:
def update_excel(transaction_data):
    """Add a new transaction to the Expenses sheet, preserving all other sheets (including Balance)."""
    try:
        # Read all sheets
        if os.path.exists(EXCEL_FILE):
            all_sheets = pd.read_excel(EXCEL_FILE, sheet_name=None)
            df = all_sheets.get('Expenses', pd.DataFrame(columns=['Date', 'Description', 'Amount', 'Category', 'Type']))
        else:
            all_sheets = {}
            df = pd.DataFrame(columns=['Date', 'Description', 'Amount', 'Category', 'Type'])
        # Create new row with date as datetime
        new_row = pd.DataFrame([{
            'Date': pd.to_datetime(transaction_data['date']),
            'Description': transaction_data['description'],
            'Amount': transaction_data['amount'],
            'Category': transaction_data['category'],
            'Type': transaction_data['type']
        }])
        # Append new row and ensure all columns exist
        df = pd.concat([df, new_row], ignore_index=True)
        # Ensure Date column is datetime type and sort
        df['Date'] = pd.to_datetime(df['Date'])
        df = df.sort_values('Date', ascending=True).reset_index(drop=True)
        # Convert dates to the desired format before saving
        df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')
        # Update Expenses sheet in all_sheets
        all_sheets['Expenses'] = df
        # Save all sheets back to Excel
        with pd.ExcelWriter(EXCEL_FILE, engine='openpyxl') as writer:
            for sheet, sheet_df in all_sheets.items():
                sheet_df.to_excel(writer, index=False, sheet_name=sheet)
        return True
    except Exception as e:
        print(f"Error updating Excel file: {e}")
        return False

In [ ]:
def get_balance():
    """Read the current balance from the Balance sheet. If not present, return None."""
    if os.path.exists(EXCEL_FILE):
        try:
            all_sheets = pd.read_excel(EXCEL_FILE, sheet_name=None)
            if 'Balance' in all_sheets:
                balance_df = all_sheets['Balance']
                if not balance_df.empty and 'Balance' in balance_df.columns:
                    return float(balance_df['Balance'].iloc[0])
        except Exception as e:
            print(f"Error reading balance: {e}")
    return None

def set_balance(new_balance):
    """Set the user's total balance in a separate Balance sheet."""
    try:
        # Read all sheets
        if os.path.exists(EXCEL_FILE):
            all_sheets = pd.read_excel(EXCEL_FILE, sheet_name=None)
        else:
            all_sheets = {}
        # Update or create Balance sheet
        balance_df = pd.DataFrame({'Balance': [new_balance]})
        all_sheets['Balance'] = balance_df
        # Write all sheets back
        with pd.ExcelWriter(EXCEL_FILE, engine='openpyxl') as writer:
            for sheet, df in all_sheets.items():
                df.to_excel(writer, index=False, sheet_name=sheet)
        return True
    except Exception as e:
        print(f"Error setting balance: {e}")
        return False

def update_balance_on_transaction(transaction_data, is_add=True):
    """Update the balance after a transaction. Add for income, subtract for expense."""
    balance = get_balance()
    if balance is None:
        print("Balance not set. Please set your balance first using 'set balance to <amount>'.")
        return False
    amount = float(transaction_data['amount'])
    if transaction_data['type'].lower() == 'income':
        balance = balance + amount if is_add else balance - amount
    else:
        balance = balance - amount if is_add else balance + amount
    return set_balance(balance)

def parse_balance_prompt(text):
    """Parse a prompt like 'set balance to 5000' and return the amount."""
    import re
    match = re.search(r'set balance to\s*([\d\.]+)', text.lower())
    if match:
        return float(match.group(1))
    return None

def create_transaction_interface():
    """Create an interactive interface for entering/removing transactions and setting/updating balance using natural language in the same text box.
    Example removal: remove $5 pizza on 25th July
    Example add: $5 on coffee and $20 on lunch
    Example set balance: set balance to 5000
    """
    text_input = widgets.Text(
        value='',
        placeholder='Enter transaction(s), removal, or balance (e.g., "$5 on coffee", "remove $5 pizza", "set balance to 5000")',
        description='Transaction/Remove/Balance:',
        style={'description_width': 'initial'},
        layout={'width': '500px'}
    )
    output = widgets.Output()

    def on_submit(b):
        with output:
            clear_output()
            if not text_input.value:
                print("Please enter a transaction, removal, or balance query.")
                return
            user_input = text_input.value.strip()
            # Set balance if prompt matches
            balance_val = parse_balance_prompt(user_input)
            if balance_val is not None:
                print(f"Setting balance to: {balance_val}")
                if set_balance(balance_val):
                    print(f"Balance successfully set to {balance_val}")
                else:
                    print("Error setting balance.")
            # Remove transaction
            elif user_input.lower().startswith(('remove', 'delete')):
                print(f"Processing removal: {user_input}")
                removals = parse_removal_prompt(user_input)
                if not removals:
                    print("Could not parse the removal prompt. Please try again.")
                    return
                print("\nParsed Transaction(s) to Remove:")
                success = True
                for i, removal_data in enumerate(removals, 1):
                    print(f"\nTransaction {i}:")
                    print(f"Date: {removal_data['date']}")
                    print(f"Description: {removal_data['description']}")
                    print(f"Amount: ${removal_data['amount']}")
                    print(f"Category: {removal_data['category']}")
                    print(f"Type: {removal_data['type']}")
                    if not remove_transaction_from_excel(removal_data):
                        print("Could not find or remove this transaction.")
                        success = False
                    else:
                        update_balance_on_transaction(removal_data, is_add=False)
                if success:
                    print(f"\nRequested transaction(s) removed from: {EXCEL_FILE}")
                else:
                    print("\nSome transactions could not be removed. Please check your prompt.")
            # Add transaction
            else:
                print(f"Processing: {user_input}")
                transactions = parse_transaction(user_input)
                if not transactions:
                    print("Could not parse the transaction(s). Please try again.")
                    return
                print("\nParsed Transaction(s):")
                success = True
                for i, transaction_data in enumerate(transactions, 1):
                    print(f"\nTransaction {i}:")
                    print(f"Date: {transaction_data['date']}")
                    print(f"Description: {transaction_data['description']}")
                    print(f"Amount: ${transaction_data['amount']}")
                    print(f"Category: {transaction_data['category']}")
                    print(f"Type: {transaction_data['type']}")
                    if not update_excel(transaction_data):
                        success = False
                        break
                    else:
                        update_balance_on_transaction(transaction_data, is_add=True)
                if success:
                    print(f"\nAll transactions successfully recorded in: {EXCEL_FILE}")
                else:
                    print("\nError recording transactions. Please try again.")
            text_input.value = ''

    submit_button = widgets.Button(
        description='Submit',
        button_style='primary'
    )
    submit_button.on_click(on_submit)

    display(widgets.VBox([text_input, submit_button, output]))

create_transaction_interface()